# Aşama 1 — Wide Residual Block’u adım adım yazma

### 1) Hedef: Ne üretmek istiyoruz?

Wide residual block’ın omurgası değişmez:
* **y=skip(x)+F(x)**
Burada:

* skip(x): ya x (identity), ya da projection (1×1 conv)

* F(x): 2 tane 3×3 conv (wide = kanalı büyütme burada)
## Kural: Pre-activation düzeni

Wide ResNet’te yaygın düzen:

* BN + ReLU → Conv (activation conv’dan önce)

* sonra yine BN + ReLU → Conv

* Dropout varsa genelde iki conv arasına koyulur.

## Aşama 1 Kod — Parça parça yazıyoruz 
* 1×1 conv = her pikselde kanal vektörüne uygulanan bir lineer dönüşüm.
Kanalları “birleştirir” çünkü yeni kanallar, eskilerin ağırlıklı toplamıdır.
“Shape düzenler” çünkü C’yi değiştirir, stride verirsek H×W’yi de değiştirir.
### 1.1) En sade WideResidualBlock (dropout yok)
* Aşağıdaki kod “çekirdek blok”tur.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WideResidualBlock(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, stride: int = 1):
        super().__init__()

        self.bn1   = nn.BatchNorm2d(in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)

        self.bn2   = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)

        self.proj = None
        if stride != 1 or in_ch != out_ch:
            self.proj = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False)

    def forward(self, x):
        # Skip
        skip = x if self.proj is None else self.proj(x)

        # F(x)
        out = self.bn1(x)
        out = F.relu(out, inplace=True)
        out = self.conv1(out)

        out = self.bn2(out)
        out = F.relu(out, inplace=True)
        out = self.conv2(out)

        # Residual sum
        out = out + skip
        return out

### 1.2) Dropout eklemek (opsiyonel)

* Dropout blok güçlü olduğu için overfitting’i azaltmak amacıyla eklenir. Tipik yer: conv1 sonrası.

In [6]:
class WideResidualBlock(nn.Module):
    def __init__(self, in_ch: int, out_ch: int, stride: int = 1, dropout_p: float = 0.0):
        super().__init__()

        self.bn1   = nn.BatchNorm2d(in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride=stride, padding=1, bias=False)

        self.dropout = nn.Dropout(p=dropout_p) if dropout_p and dropout_p > 0 else None

        self.bn2 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, stride=1, padding=1, bias=False)

        self.proj = None
        if stride != 1 or in_ch != out_ch:
            self.proj = nn.Conv2d(in_ch, out_ch, 1, stride=stride, bias=False)

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)

        out = self.bn1(x)
        out = F.relu(out, inplace=True)
        out = self.conv1(out)

        if self.dropout is not None:
            out = self.dropout(out)

        out = self.bn2(out)
        out = F.relu(out, inplace=True)
        out = self.conv2(out)

        return out + skip

### 1.3) “Çalışıyor mu?” shape testi (kritik)

In [7]:
x = torch.randn(2, 64, 56, 56)

blk_id = WideResidualBlock(64, 64, stride=1, dropout_p=0.0)
y1 = blk_id(x)  # [2, 64, 56, 56]

blk_proj = WideResidualBlock(64, 128, stride=2, dropout_p=0.0)
y2 = blk_proj(x)  # [2, 128, 28, 28]

print(x.shape, y1.shape, y2.shape)


torch.Size([2, 64, 56, 56]) torch.Size([2, 64, 56, 56]) torch.Size([2, 128, 28, 28])


----
----
----
----

# Aşama 2-) Modeli profesyonel hale getirelim

### 2.1) Stage builder: “N tane blok” üretmek ✅

Kural:

* Stage’in ilk bloğu stride alır (downsample gerekiyorsa)

- Sonrakiler stride=1

* İlk blokta in_ch -> out_ch dönüşümü olur

In [8]:
def make_wide_layer(in_ch:int,out_ch:int,num_block:int,stride:int,dropout_p:float):
    layers = []
    layers.append(WideResidualBlock(in_ch,out_ch,stride=stride,dropout_p=dropout_p))
    for _ in range(1,num_block):
        layers.append(WideResidualBlock(out_ch,out_ch,stride=1,dropout_p=dropout_p))
    return nn.Sequential(*layers)

### `make_wide_layer(...)` Ne Yapıyor? (Stage / Katman Grubu Üretici)

Bu fonksiyonun görevi şudur:

> **Tek bir “stage” (blok grubu) oluşturmak**  
> Yani arka arkaya `num_block` tane `WideResidualBlock` dizmek ve bunları **tek bir modül** gibi döndürmek.



#### 1) Girdi Parametreleri Ne Anlama Geliyor?

- **`in_ch`**: Stage’e giren kanal sayısı  
- **`out_ch`**: Stage’den çıkan kanal sayısı (bu stage’in “genişliği”)  
- **`num_block`**: Bu stage içinde kaç tane `WideResidualBlock` olacağı  
- **`stride`**: Stage’in *ilk bloğunun* stride’ı (downsample burada yapılır)  
- **`dropout_p`**: Block içindeki dropout oranı

####  2) Neden İlk Blok Farklı?

Stage içinde **sadece ilk blok** şu işleri yapabilir:

- `in_ch -> out_ch` kanal dönüşümü
- `stride=2` ise `H×W` downsample (boyutu küçültme)

Bu yüzden ilk blok:

```python
WideResidualBlock(in_ch, out_ch, stride=stride)
```
şeklinde kurulur.



### `layers` Listesi Ne Oluyor?

Fonksiyon sonunda layers şu hale gelir:

* blok: in_ch -> out_ch (stride = verilen stride)

* blok: out_ch -> out_ch (stride=1)

* ...

* num_block. blok: out_ch -> out_ch (stride=1)
Yani:
```bash
[ Block(in_ch→out_ch, stride=stride),
  Block(out_ch→out_ch, stride=1),
  Block(out_ch→out_ch, stride=1),
  ... (toplam num_block adet) ]
```

### `nn.Sequential` demek:

* Bu blokları sırayla çalıştır, tek bir modül gibi davran.



>**make_wide_layer, num_block tane WideResidualBlock'u arka arkaya dizip,ilk blokta kanal/çözünürlük geçişini yapacak şekilde bir stage oluşturur ve bunu nn.Sequential olarak döndürür.**

## 2.1-) Gerçek backbone: Stem + 3 stage ✅

* Bu backbone, istersek classifier’a, istersek detection neck/head’e feature üretir.

In [11]:
class WideResNetBackbone(nn.Module):
    def __init__(self, depth:int = 128,widen_factor:int = 2,dropout_p :float = 0.0,in_channels:int = 3):
        super().__init__()
        assert (depth-4) % 6 == 0 , "WRN için depth genelde 6n+4 formatındadır"
        n = (depth-4) // 6

        base = 16
        ch1 = base * widen_factor
        ch2 = base * 2 * widen_factor
        ch3 = base * 4 * widen_factor

        self.stem = nn.Sequential(
            nn.Conv2d(in_channels,base,kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(base),
            nn.ReLU(inplace=True),
            )
        
        self.stage1 = make_wide_layer(base,ch1,num_block=n,stride=1,dropout_p=dropout_p)
        self.stage2 = make_wide_layer(ch1,ch2,num_block=n,stride=2,dropout_p=dropout_p)
        self.stage3 = make_wide_layer(ch2,ch3,num_block=n,stride=2,dropout_p=dropout_p)

        self.bn = nn.BatchNorm2d(ch3)
        self.out_channels = ch3

    def forward(self,x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.bn(x)
        x = F.relu(x,inplace=True)
        return x

In [ ]:
class WideResNetClassifier(nn.Module):
    def __init__(self, num_classes: int, depth: int = 28, widen_factor: int = 2, dropout_p: float = 0.0):
        super().__init__()
        self.backbone = WideResNetBackbone(depth=depth, widen_factor=widen_factor, dropout_p=dropout_p)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(self.backbone.out_channels, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x).flatten(1)  # [B,C,1,1] -> [B,C]
        x = self.fc(x)
        return x

## Kullanım: “Gerçekten model oldu mu?” (shape test) ✅

In [ ]:
model = WideResNetClassifier(num_classes=10, depth=28, widen_factor=2, dropout_p=0.0)
x = torch.randn(4, 3, 32, 32)
y = model(x)
print("logits:", y.shape)  # [4,10]

feat = model.backbone(x)
print("feat:", feat.shape)  # [4, C, H', W'] -> WRN-28-2 için C=128, H'=8, W'=8 (32->16->8)

logits: torch.Size([4, 10])
feat: torch.Size([4, 128, 8, 8])
